In [152]:
import pandas as pd

In [153]:
path_gc = 'GCNetwork.txt'
path_gnomad = "gnomad.v2.1.1.all_lofs.txt.bgz"

In [154]:
gnomadlof = pd.read_csv(path_gnomad, delim_whitespace=True, compression='gzip', low_memory=False)
gc = pd.read_csv(path_gc, delim_whitespace=True)

In [155]:
genes = []
df = gnomadlof.eq(gc)


In [156]:
True in df['pos']

False